In [11]:
# 必要なライブラリのインポート
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine

# PostgreSQLデータベースへの接続設定
db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '54322',
    'database': 'postgres'
}

# SQLAlchemyエンジンの作成
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# データの読み込み
yahoomap_crawds_query = "SELECT * FROM yahoomap_crawds"
event_details_query = "SELECT * FROM event_details"
weather_history_hour_query = "SELECT * FROM weather_history_hour"

yahoomap_crawds = pd.read_sql(yahoomap_crawds_query, engine)
event_details = pd.read_sql(event_details_query, engine)
weather_history_hour = pd.read_sql(weather_history_hour_query, engine)

In [15]:
# 祝日リスト
holiday_list = ["2024-01-01","2024-01-08","2024-02-11","2024-02-12","2024-02-23","2024-03-20","2024-04-29","2024-05-03","2024-05-04","2024-05-05","2024-05-06","2024-07-15","2024-08-11","2024-08-12","2024-09-16","2024-09-22","2024-09-23","2024-10-14","2024-11-03","2024-11-04","2024-11-23"]

# 必要なライブラリのインポート
psycopg2やSQLAlchemyなどの必要なライブラリをインポートします。

In [1]:
# 必要なライブラリのインポート
import psycopg2
from sqlalchemy import create_engine

# データベース接続の設定
データベースのホスト、ポート、ユーザー名、パスワード、データベース名などの接続情報を設定します。

In [2]:
# データベース接続の設定

# データベース接続情報の設定
db_config = {
    'host': 'localhost',  # データベースのホスト名またはIPアドレス
    'port': '54322',  # データベースのポート番号
    'user': 'postgres',  # データベースのユーザー名
    'password': 'postgres',  # データベースのパスワード
    'dbname': 'postgres'  # データベース名
}

# psycopg2を使用して接続文字列を作成
conn_string = f"host={db_config['host']} port={db_config['port']} user={db_config['user']} password={db_config['password']} dbname={db_config['dbname']}"

# SQLAlchemyを使用して接続URLを作成
engine_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"

# psycopg2を使用してデータベースに接続
conn = psycopg2.connect(conn_string)

# SQLAlchemyを使用してエンジンを作成
engine = create_engine(engine_url)

# データベースへの接続
設定した接続情報を使用してPostgreSQLデータベースに接続します。

In [3]:
# データベースへの接続
# 設定した接続情報を使用してPostgreSQLデータベースに接続します。

# psycopg2を使用して接続を確認
try:
    cur = conn.cursor()
    cur.execute("SELECT version();")
    db_version = cur.fetchone()
    print(f"PostgreSQL database version: {db_version}")
    cur.close()
except Exception as error:
    print(f"Error: {error}")
finally:
    if conn is not None:
        conn.close()

# SQLAlchemyを使用して接続を確認
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT version();")
        for row in result:
            print(f"PostgreSQL database version: {row[0]}")
except Exception as error:
    print(f"Error: {error}")

PostgreSQL database version: ('PostgreSQL 14.14 (Debian 14.14-1.pgdg120+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Error: Not an executable object: 'SELECT version();'


# クエリの実行
接続したデータベースに対してSQLクエリを実行し、結果を取得します。

In [6]:
# 必要なライブラリのインポート
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine

# PostgreSQLデータベースへの接続設定
db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '54322',
    'database': 'postgres'
}

# SQLAlchemyエンジンの作成
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# データの読み込み
yahoomap_crawds_query = "SELECT * FROM db.public.yahoomap_crawds"
event_details_query = "SELECT * FROM db.public.event_details"

yahoomap_crawds = pd.read_sql(yahoomap_crawds_query, engine)
event_details = pd.read_sql(event_details_query, engine)

# データの前処理
# 日付と時間の変換
yahoomap_crawds['date'] = pd.to_datetime(yahoomap_crawds['date'])

# 時間の文字列を整数に変換
yahoomap_crawds['hour'] = yahoomap_crawds['hour'].apply(lambda x: int(x.split(':')[0]))

# 必要なカラムの選択
yahoomap_crawds = yahoomap_crawds[['place', 'date', 'weekday', 'hour', 'crawds_level']]
event_details = event_details[['place', 'date', 'num_of_people']]

# データのマージ
data = pd.merge(yahoomap_crawds, event_details, on=['place', 'date'], how='left')

# 欠損値の処理
data['num_of_people'].fillna(0, inplace=True)

# 特徴量とターゲットの分割
X = data[['place', 'weekday', 'hour', 'num_of_people']]
y = data['crawds_level']

# カテゴリ変数のエンコーディング
X = pd.get_dummies(X, columns=['place'], drop_first=True)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMデータセットの作成
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# モデルのパラメータ設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# モデルの訓練
model = lgb.train(params, train_data, valid_sets=[train_data, test_data], num_boost_round=1000, early_stopping_rounds=10)

# 予測
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# 評価
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

ValueError: invalid literal for int() with base 10: '20:00'